In [1]:
from traceback import print_exc

In [16]:
# a graph class, based on map, offer a general class for graphs/networks
class graph:
    # graph class initializer, by giving a dictionary,
    # dont give egdes with capacity 0, otherwise the fails happens
    def __init__(self, gdict=None):
        if (gdict == None):
            self.gdict = {}
        else:
            self.gdict = gdict

    
    # debugging use only, return the dict inside the graph
    def get_dict(self):
        return self.gdict


    # get list of keys in the graph
    def key_list(self):
        res = []
        for key in self.gdict.keys():
            res.append(key)
        return res

    
    # get edges coming out from the given vertex
    # return a list of ['vertex', weight] pairs of all edges coming out of the node
    def get_edges(self, vertex):
        return self.gdict.get(vertex)

    
    # get edges node coming out from the given vertex
    def get_edges_node(self, vertex):
        res = []
        all_edges = self.get_edges(vertex)
        if (all_edges == None):
            return []
        for i in range(0, len(all_edges)):
            res.append(all_edges[i][0])
        return res

    
    # check if an edge exist
    def edge_exist(self, start, end):
        if (start not in self.key_list()):
            raise Exception('The given start node not in the graph')
        elif (end not in self.get_edges_node(start)):
            return False
        return True

    
    # get capacity of an edge starting from the start node to the target node
    def get_capacity(self, start_node, target_node):
        edge_weight_pairs = self.get_edges(start_node)
        for e in edge_weight_pairs:
            if (e[0] == target_node):
                return e[1]
        raise Exception('No edges from start node to target node matches the requirements')

    
    # determine max flow through the given path
    # path in form of list from start node to terminal node
    def max_flow_over_path(self, path):
        if (len(path) < 2):
            raise Exception('Invalid path, node less than 2')

        limit = self.get_capacity(path[0], path[1])
        for i in range(0, len(path) - 1):
            curr_weight = self.get_capacity(path[i], path[i+1])
            if (curr_weight < limit):
                limit = curr_weight

        return limit

    
    # update an edge with given capacity
    # handle cases if there is no start or no existing eadge. create new edge if needed
    # (void function)
    def update_edge(self, start, end, capacity):
        if (capacity != 0):
            old_edges_pair = self.get_edges(start)
            try:
                if (self.edge_exist(start, end)):
                    old_value = [end, self.get_capacity(start, end)]
                    old_edges_pair[old_edges_pair.index(old_value)] = [end, capacity]
                else:
                    print('Alert: the edge not here, creating a new one')
                    old_edges_pair.append([end, capacity])
            except Exception as e:
                print('Alert: start node may not be there, creating one')
                print(e)
                old_edges_pair = [[end, capacity]]
            finally:
                self.gdict.update({start: old_edges_pair})
        else:
            old_edges_pair = self.get_edges(start)
            old_value = [end, self.get_capacity(start, end)]
            old_edges_pair.pop(old_edges_pair.index(old_value))
            self.gdict.update({start: old_edges_pair})


    # update augmented path when decide let a maximum flow go through it,
    # the function returns the max flow went through this augmented path during current iteration
    def update_augmented_path(self, augmented_path):
        max_flow = self.max_flow_over_path(augmented_path)
        for i in range(0, len(augmented_path) - 1):
            start = augmented_path[i]
            end = augmented_path[i + 1]
            old_cap = self.get_capacity(start, end)
            new_cap = old_cap - max_flow
            # update the new capacity after we ran a max flow on the route
            self.update_edge(start, end, new_cap)
            # set the residue edge with a capacity of the maxflow
            self.update_edge(end, start, max_flow)
        return max_flow
    


In [3]:
# generate a list of n repeated elements
def n_node(n, name):
    res = []
    for i in range(0, n):
        res.append(name)
    return res

# convert a path in the form of maps{node: parent} to a list form
def path_to_list(path, source, terminal):
    res = [terminal]
    node = terminal
    while (node != source):
        node = path.get(node)
        res.append(node)

    res.reverse()
    return res


# algorithm to find a shortest path tree
# g: a graph object
def find_shortest_path(g, source, terminal):
    visited = [source]
    all_paths = {}

    # queue stores each level
    # last_node stores the parent of the nodes in queue one by one
    queue = g.get_edges_node(source)

    last_node = n_node(len(queue), source)

    while (len(queue) > 0):
        curr_node = queue.pop()
        curr_parent = last_node.pop()
        if (curr_node not in visited):
            visited.append(curr_node)
            all_paths.update({curr_node:curr_parent})
            if (curr_node == terminal):
                return path_to_list(all_paths, source, terminal)
            curr_child = g.get_edges_node(curr_node)
            queue = curr_child + queue
            last_node = n_node(len(curr_child), curr_node) + last_node

    # didn't find any path to terminal
    return None




In [4]:
# test shortest path algo with cases
# example_graph1 = {'s':[['b', 3], ['c', 2]], 
#                 'b': [['c', 3], ['d', 2]],
#                 'c': [['e', 3]],
#                 'd': [['t', 3]],
#                 'e': [['d', 3]],
#                 't': []}

# example_graph2 = {'s':[['1', 3], ['3', 2]], 
#                 '3': [['7', 3], ['4', 2]],
#                 '1': [['2', 3]],
#                 '7': [['4', 3], ['6', 2]],
#                 '4': [['6', 3], ['t', 2], ['3', 2]],
#                 '6': [['t', 3]],
#                 '2': [],
#                 't': []}

# temp1 = find_shortest_path(graph(example_graph1), 's', 't')
# temp2 = find_shortest_path(graph(example_graph2), 's', 't')
# print(temp1)
# print(temp2)

In [5]:
# class function testing purpose

# print(graph(example_graph1).get_capacity('c', 'e'))
# print(graph(example_graph1).max_flow_over_path(temp1))
# temp = graph()
# temp.update_edge('s', 'b', 1)
# print(temp.get_dict())
# temp.update_edge('s', 'a', 2)
# print(temp.get_dict())
# temp.update_edge('s', 'a', 0)
# print(temp.get_dict())


In [6]:
# requires distinct node names in graph
def ek_maxflow(g, source, terminal):
    all_augmented_paths = []
    total_max_flow = 0
    curr_sp = find_shortest_path(g, source, terminal)
    while (curr_sp != None):
        all_augmented_paths.append(curr_sp)
        total_max_flow += g.update_augmented_path(curr_sp)
        curr_sp = find_shortest_path(g, source, terminal)
        
    
    return total_max_flow, all_augmented_paths



In [7]:
example_graph3 = {'s':[['1', 16], ['2', 13]], 
                '1': [['2', 10], ['3', 12]],
                '2': [['1', 4],['4', 14]],
                '3': [['2', 9], ['t', 20]],
                '4': [['3', 7], ['t', 4]],
                't': []}
                
ex3_res = ek_maxflow(graph(example_graph3), 's', 't')
ex3_res

Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one


(23, [['s', '2', '4', 't'], ['s', '1', '3', 't'], ['s', '2', '4', '3', 't']])

In [8]:
example_graph4 = {'s':[['0', 1], ['2', 1], ['5', 1]], 
                '0': [['1', 1], ['3', 1], ['4', 1], ['6', 1]],
                '2': [['3', 1], ['4', 1], ['6', 1]],
                '5': [['6', 1]],
                '1': [['t', 1]],
                '3': [['t', 1]],
                '4': [['t', 1]],
                '6': [['t', 1]],
                't': []}
ex4_res = ek_maxflow(graph(example_graph4), 's', 't')
ex4_res

Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one


(3, [['s', '5', '6', 't'], ['s', '2', '4', 't'], ['s', '0', '3', 't']])

In [21]:
# test algo on the old mini exam, all test passed
example_graph5 = {'s':[['m1', 4], ['m2', 4], ['m3', 2], ['m4', 2], ['m5', 2], ['m6', 2], ['m7', 2], ['m8', 2], ['m9', 2], ['m10', 2], ['m11', 2], ['m12', 2], ['m13', 4], ['m14', 2], ['m15', 2]], 
                'm1': [['p1', 4], ['p2', 4]],
                'm2': [['p1', 4], ['p3', 4]],
                'm3': [['p1', 2],['p4', 2]],
                'm4': [['p1', 2],['p5', 2]],
                'm5': [['p1', 2],['p6', 2]],
                'm6': [['p2', 2],['p3', 2]],
                'm7': [['p2', 2],['p4', 2]],
                'm8': [['p2', 2],['p5', 2]],
                'm9': [['p2', 2],['p6', 2]],
                'm10': [['p3', 2],['p4', 2]],
                'm11': [['p3', 2],['p5', 2]],
                'm12': [['p3', 2],['p6', 2]],
                'm13': [['p4', 2],['p5', 2]],
                'm14': [['p4', 2],['p6', 2]],
                'm15': [['p5', 2],['p6', 2]],
                'p1': [['t', 10]],
                'p2': [['t', 9]],
                'p3': [['t', 9]],
                'p4': [['t', 4]],
                'p5': [['t', 4]],
                't': []}
ex5_res = ek_maxflow(graph(example_graph5), 's', 't')
print(ex5_res)


example_graph6 = {'s':[ ['m3', 2], ['m4', 2], ['m6', 2], ['m7', 2], ['m8', 2], ['m10', 2], ['m11', 2]], 
                'm3': [['p1', 2],['p4', 2]],
                'm4': [['p1', 2],['p5', 2]],
                'm6': [['p2', 2],['p3', 2]],
                'm7': [['p2', 2],['p4', 2]],
                'm8': [['p2', 2],['p5', 2]],
                'm10': [['p3', 2],['p4', 2]],
                'm11': [['p3', 2],['p5', 2]],
                'p2': [['t', 1]],
                'p3': [['t', 1]],
                'p4': [['t', 6]],
                'p5': [['t', 6]],
                't': []}

ex6_res = ek_maxflow(graph(example_graph6), 's', 't')
print(ex6_res)

example_graph7 = {'s':[['m3', 2], ['m4', 2], ['m6', 2], ['m7', 2], ['m8', 2], ['m10', 2], ['m11', 2]], 
                'm3': [['p1', 2],['p4', 2]],
                'm4': [['p1', 2],['p5', 2]],
                'm6': [['p2', 2],['p3', 2]],
                'm7': [['p2', 2],['p4', 2]],
                'm8': [['p2', 2],['p5', 2]],
                'm10': [['p3', 2],['p4', 2]],
                'm11': [['p3', 2],['p5', 2]],
                'p4': [['t', 3]],
                'p5': [['t', 3]],
                't': []}
ex7_res = ek_maxflow(graph(example_graph7), 's', 't')
print(ex7_res)


Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the

In [24]:
# actual problem part 1
p1_graph = {'s':[['yp', 3], ['yd', 3], ['yc', 3], ['pd', 3], ['pc', 3], ['hd', 3], ['hc', 3]], 
                'yp': [['y', 3], ['p', 3]],
                'yh': [['y', 3], ['h', 3]],
                'yd': [['y', 3],['d', 3]],
                'yc': [['y', 3],['c', 3]],
                'ph': [['p', 3],['h', 3]],
                'pd': [['p', 3],['d', 3]],
                'pc': [['p', 3],['c', 3]],
                'hd': [['h', 3],['d', 3]],
                'hc': [['h', 3],['c', 3]],
                'dc': [['d', 3],['c', 3]],
                'p': [['t', 1]],
                'h': [['t', 4]],
                'd': [['t', 6]],
                'c': [['t', 10]],
                't': []}
p1_res = ek_maxflow(graph(p1_graph), 's', 't')
print(p1_res)



Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
(19, [['s', 'hc', 'c', 't'], ['s', 'hd', 'd', 't'], ['s', 'pc', 'c', 't'], ['s', 'pd', 'd', 't'], ['

In [25]:
# actual problem part 2
p2_graph = {'s':[['yp', 3], ['yd', 3], ['pd', 3], ['pb', 3], ['hd', 3], ['hb', 3], ['db', 3]], 
                'yp': [['y', 3], ['p', 3]],
                'yh': [['y', 3], ['h', 3]],
                'yd': [['y', 3],['d', 3]],
                'yb': [['y', 3],['b', 3]],
                'ph': [['p', 3],['h', 3]],
                'pd': [['p', 3],['d', 3]],
                'pb': [['p', 3],['b', 3]],
                'hd': [['h', 3],['d', 3]],
                'hb': [['h', 3],['b', 3]],
                'db': [['d', 3],['b', 3]],
                'y': [['t', 2]],
                'p': [['t', 3]],
                'h': [['t', 6]],
                'd': [['t', 8]],
                'b': [['t', 14]],
                't': []}
p2_res = ek_maxflow(graph(p2_graph), 's', 't')
print(p2_res)



Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
(21, [['s', 'db', 'b', 't'], ['s', 'hb', 'b', 't'], ['s', 'hd', 'd', 't'], ['s', 'pb', 'b', 't'], ['s', 'pd', 'd', 't'], ['s', 'yd', 'd', 't'], [

In [26]:
# actual problem part 3
p3_graph = {'s':[['yp', 3], ['yd', 3], ['pd', 3], ['pb', 3], ['hd', 3], ['hb', 3], ['db', 3], ['cx', 12]], 
                'yp': [['y', 3], ['p', 3]],
                'yd': [['y', 3],['d', 3]],
                'pd': [['p', 3],['d', 3]],
                'pb': [['p', 3],['b', 3]],
                'hd': [['h', 3],['d', 3]],
                'hb': [['h', 3],['b', 3]],
                'db': [['d', 3],['b', 3]],
                'cx': [['y', 3],['p', 3], ['h', 3], ['d', 3],['b', 3]],
                'y': [['t', 5]],
                'p': [['t', 6]],
                'h': [['t', 9]],
                'd': [['t', 11]],
                'b': [['t', 2]],
                't': []}
p3_res = ek_maxflow(graph(p3_graph), 's', 't')
print(p3_res)



Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the edge not here, creating a new one
Alert: the

In [29]:
print(f'Part 1 network max flow: {p1_res[0]}')
print(f'All augmented Paths used: {p1_res[1]}')
print()
print(f'Part 2 network max flow: {p2_res[0]}')
print(f'All augmented Paths used: {p2_res[1]}')
print()
print(f'Part 3 network max flow: {p3_res[0]}')
print(f'All augmented Paths used: {p3_res[1]}')

Part 1 network max flow: 19
All augmented Paths used: [['s', 'hc', 'c', 't'], ['s', 'hd', 'd', 't'], ['s', 'pc', 'c', 't'], ['s', 'pd', 'd', 't'], ['s', 'yc', 'c', 't'], ['s', 'yp', 'p', 't'], ['s', 'yd', 'd', 'hd', 'h', 't']]

Part 2 network max flow: 21
All augmented Paths used: [['s', 'db', 'b', 't'], ['s', 'hb', 'b', 't'], ['s', 'hd', 'd', 't'], ['s', 'pb', 'b', 't'], ['s', 'pd', 'd', 't'], ['s', 'yd', 'd', 't'], ['s', 'yd', 'y', 't'], ['s', 'yp', 'p', 't']]

Part 3 network max flow: 33
All augmented Paths used: [['s', 'cx', 'b', 't'], ['s', 'cx', 'd', 't'], ['s', 'cx', 'h', 't'], ['s', 'cx', 'p', 't'], ['s', 'cx', 'y', 't'], ['s', 'db', 'd', 't'], ['s', 'hb', 'h', 't'], ['s', 'hd', 'd', 't'], ['s', 'pb', 'p', 't'], ['s', 'pd', 'd', 't'], ['s', 'yd', 'y', 't'], ['s', 'yp', 'y', 't'], ['s', 'pd', 'd', 'hd', 'h', 't'], ['s', 'yp', 'y', 'yd', 'd', 'hd', 'h', 't']]
